In [1]:
import pandas as pd

In [2]:
# загружаем файл
df_l2b = pd.read_csv('L2B.txt', sep = ';')

In [3]:
# смотрим данные
df_l2b.head()

,Направление,Поиски,Сегменты,L2B
0,MOW-AER,5458010,1886,2893
1,MOW-LED,3728800,102,36556
2,AER-MOW,2741115,673,4072
3,MOW-SIP,2634666,663,3973
4,MOW-IST,2588738,43,60203


In [4]:
# смотрим есть ли пропуски в данных
df_l2b.isna().sum()

Направление    0
Поиски         0
Сегменты       0
L2B            0
dtype: int64

In [5]:
# смотрим какие значения встречаются в сегменты
df_l2b['Сегменты'].unique()

array([1886,  102,  673,  663,   43,  169,   14,  155,  908,  417,  125,
        489,  608,   41,   13,  132,  579,  328,   74,   10,   75,   37,
          3, 1052,    1,  477,   42,   36,    0,  196,   31,   60,    6,
        768,   11,   68,   59,    5,   19,  101,   81,   23,   64,  345,
         18,   30,    4,   92,   83,    7,  202,  136,   20,  126,   27,
         15,    8,   40,    9,   32,   26,   44,   16,   29,   70,   79,
         62,   28,    2,   34,   12,   58,   56,   21,  187,   55,  263,
         24,   72,   51,  178,  218,   17,  117,  277,   96,  140,   80,
         71,   82,  122,  129,   50,  201,   38,   22,  120,  299,   54,
         88,  118,  113,  211,   47,  189,  314,  128,  319,   46,   95,
        163,   33,   63,  245,  145,  147,   48,   77,   76,  141,   25,
         52,   66,  138,   39,   35,   90,   45,   69], dtype=int64)

In [6]:
# удаляем строки, в которых сегменты = 0
df_l2b_corr = df_l2b[df_l2b['Сегменты'] != 0].copy()
df_l2b_corr.head()

,Направление,Поиски,Сегменты,L2B
0,MOW-AER,5458010,1886,2893
1,MOW-LED,3728800,102,36556
2,AER-MOW,2741115,673,4072
3,MOW-SIP,2634666,663,3973
4,MOW-IST,2588738,43,60203


In [7]:
def trim_col_from (row):
    '''Отбирает часть из'''
    return row['Направление'].split('-')[0]
def trim_col_to (row):
    '''Отбирает часть в'''
    return row['Направление'].split('-')[1]

In [20]:
#получаем новые столбцы
df_l2b_corr['from'] = df_l2b_corr.apply(trim_col_from, axis = 1)
df_l2b_corr['to'] = df_l2b_corr.apply(trim_col_to, axis = 1)
df_l2b_corr['l2b'] = round(df_l2b_corr['Поиски']/df_l2b_corr['Сегменты']).astype(int)
df_l2b_corr.head(5)


,Направление,Поиски,Сегменты,L2B,from,to,l2b
0,MOW-AER,5458010,1886,2893,MOW,AER,2894
1,MOW-LED,3728800,102,36556,MOW,LED,36557
2,AER-MOW,2741115,673,4072,AER,MOW,4073
3,MOW-SIP,2634666,663,3973,MOW,SIP,3974
4,MOW-IST,2588738,43,60203,MOW,IST,60203


In [29]:
#Найти показатель L2B для всех направлений, в которых встречается код MOW. При расчете игнорировать направления без Сегментов.
result = df_l2b_corr[(df_l2b_corr['from'] == 'MOW' )| (df_l2b_corr['to'] == 'MOW')].iloc[:,[0, 6]]
result.head()

,Направление,l2b
0,MOW-AER,2894
1,MOW-LED,36557
2,AER-MOW,4073
3,MOW-SIP,3974
4,MOW-IST,60203


In [30]:
tt = result['l2b'].sum()
print(tt)

4311934


In [41]:
#Определить топ-3 городов с наибольшим суммарным значением L2B по всем направлениям, где они представлены.
prep = df_l2b_corr.iloc[:, [4, 6]].copy()
sum_l2b_from = prep.groupby(by = 'from').sum().reset_index().copy()
sum_l2b_from

,from,l2b
0,AAQ,139104
1,ADA,298
2,AER,263836
3,ARH,23198
4,ASF,5823
...,...,...
90,VOG,5943
91,VOZ,9393
92,VVO,14628
93,YKS,32392


In [42]:
prep1 = df_l2b_corr.iloc[:, [5, 6]].copy()
sum_l2b_to = prep1.groupby(by = 'to').sum().reset_index().copy()
sum_l2b_to

,to,l2b
0,AAQ,111986
1,ABA,4665
2,ADA,34
3,AER,293648
4,ARH,15239
...,...,...
104,UUS,1504
105,VOG,78260
106,VOZ,42563
107,VVO,38797


In [44]:
all_l2b = sum_l2b_from.merge(sum_l2b_to, left_on = 'from', right_on = 'to')
all_l2b

,from,l2b_x,to,l2b_y
0,AAQ,139104,AAQ,111986
1,ADA,298,ADA,34
2,AER,263836,AER,293648
3,ARH,23198,ARH,15239
4,BAX,52783,BAX,86067
...,...,...,...,...
80,UUS,800,UUS,1504
81,VOG,5943,VOG,78260
82,VOZ,9393,VOZ,42563
83,VVO,14628,VVO,38797


In [50]:
all_l2b['total'] = all_l2b['l2b_x'] + all_l2b['l2b_y']
top3 = all_l2b.sort_values(by = 'total', ascending = False).head(3).iloc[:, [0,4]]
top3

,from,total
40,MOW,4311934
35,LED,2145701
31,KRR,1271874
